In [30]:
import os
import gc
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
from data import ModelNet40
from data import Bunny
import numpy as np
from torch.utils.data import DataLoader
from model import PRNet
from types import SimpleNamespace
import seaborn as sns
from tqdm import tqdm

from nuscenesHelper.utils import scale_to_range, filter_pointcloud
from nuscenesHelper import LidarDataset
from nuscenesHelper import NuScenesHelper
from nuscenes.nuscenes import NuScenes, NuScenesExplorer

In [31]:
args = SimpleNamespace(
    emb_nn='dgcnn', 
    attention='transformer', 
    head='svd',
    svd_on_gpu=True,
    n_emb_dims=512,
    n_blocks=1,
    n_heads=4,
    n_iters=3,
    discount_factor=0.9,
    n_ff_dims=1024,
    n_keypoints=512,
    temp_factor=100,
    cat_sampler='gumbel_softmax',
    dropout=0,
    batch_size=6,
    test_batch_size=6,
    epochs=100,
    cycle_consistency_loss=0.1,
    feature_alignment_loss=0.1,
    n_points=1024,
    rot_factor=4,
    exp_name="exp",
    n_subsampled_points=768,
    model_path=""
)
def _init_(args):
    if not os.path.exists('checkpoints'):
        os.makedirs('checkpoints')
    if not os.path.exists('checkpoints/' + args.exp_name):
        os.makedirs('checkpoints/' + args.exp_name)
    if not os.path.exists('checkpoints/' + args.exp_name + '/' + 'models'):
        os.makedirs('checkpoints/' + args.exp_name + '/' + 'models')
    os.system('cp main.py checkpoints' + '/' + args.exp_name + '/' + 'main.py.backup')
    os.system('cp model.py checkpoints' + '/' + args.exp_name + '/' + 'model.py.backup')
    os.system('cp data.py checkpoints' + '/' + args.exp_name + '/' + 'data.py.backup')
_init_(args)

In [32]:
net = PRNet(args).cuda()
model_path = 'checkpoints' + '/' + args.exp_name + '/models/model.best.t7'
state = torch.load(model_path)
net.set_state(state['model_state_dict'])

In [22]:
test_dataset = ModelNet40(num_points=args.n_points,
                          num_subsampled_points=args.n_subsampled_points,
                          partition='test', gaussian_noise=False,
                          unseen=True, rot_factor=args.rot_factor)
test_loader = DataLoader(test_dataset, 
                         batch_size=args.test_batch_size, 
                         shuffle=False, 
                         drop_last=False)
net.eval()
with torch.no_grad():
    for data in test_loader:
        src, tgt, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba = [d.cuda() for d in data]
        rotation_ab_pred, translation_ab_pred = net.predict(src, tgt, n_iters=6)
                
        print("True transformation:")
        print(rotation_ab)
        print(translation_ab)
        print("------------------------")
        print("Predicted transformation:")
        print(rotation_ab_pred)
        print(translation_ab_pred)
        print("------------------------")
        break

In [26]:
print(src.shape, tgt.shape)
print(rotation_ab.shape, translation_ab.shape)

torch.Size([6, 3, 768]) torch.Size([6, 3, 768])
torch.Size([6, 3, 3]) torch.Size([6, 3])


In [2]:
nusc = NuScenes(version='v1.0-trainval', dataroot='/datasets_master/nuscenes', verbose=True)
nuscenesHelper = NuScenesHelper(nusc)

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 40.4 seconds.
Reverse indexing ...
Done reverse indexing in 17.9 seconds.


In [33]:
test_scenes = nuscenesHelper.get_small_validation_set()
dataset = LidarDataset(nusc, test_scenes, skip=(1, 1), n_rounds=1, get_colors=False)

100%|██████████| 20/20 [00:01<00:00, 13.05it/s]

7783 samples loaded


In [34]:
net.eval()
with torch.no_grad():
    test_loader = DataLoader(dataset, 
                         batch_size=args.test_batch_size, 
                         shuffle=False, 
                         drop_last=False)
    for data in tqdm(test_loader, position=0, leave=True):
        src = data['source_points'].float().cuda() / 35.0
        target = data['target_points'].cuda() / 35.0
        
        pred_r, pred_t = net.predict(src, target, n_iters=6)
        pred_t *= 35.0
        
        pred_r = pred_r.cpu().detach().numpy()
        pred_t = pred_t.cpu().detach().numpy()
        true_r = data['rotation'].numpy()
        true_t = data['translation'].numpy()
        token1s = data['token1']
        token2s = data['token2']
        
        for i in range(len(token1s)):
            token1 = token1s[i]
            token2 = token2s[i]
            res = {
                'true_rotation': true_r[i],
                "true_translation": true_t[i],
                "predicted_rotation": pred_r[i],
                "predicted_translation": pred_t[i],
                "token1": token1,
                "token2": token2
            }
            filename = token1 + '_' + token2 + '.npy'
            np.save(os.path.join('/root/no_backup/prnet_pretrained_modelnet40', filename), res) 
#             print("Saved: {}".format(filename))
                

100%|██████████| 1298/1298 [27:29<00:00,  1.27s/it]
